note: exercises only to validate inference time, and for inference time improvement using parallelization techniques. So many wrong practices for training, and testing will be done (e.g. purposely diplicating rows to virtually increase the dataset, and testing on the same training set)

In [ ]:
import os
import sys
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(repo_root)

import pandas as pd
import time
from utils import measure_time_decorator
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import joblib
import datetime



NUM_ROWS = int(1e6) # Number of rows to read from the dataset
N_JOBS_MODEL = os.cpu_count() # Number of jobs supported within the model itself
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
DATASET_PATH = os.path.join(ROOT_DIR, "data", "healthcare_noshows_appointments.csv")
MODELSAVE_PATH = os.path.join(ROOT_DIR, "sa")

def pipeline(X,model):

    categorical_features = [col for col in X.columns if X[col].dtype == 'category']
    numeric_features = [col for col in X.columns if X[col].dtype in ['int64', 'float64', 'bool']]

    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(handle_unknown='ignore'), ['Gender', 'Neighbourhood']),
            ('num', StandardScaler(), numeric_features)
        ]
    )
    modelwpipeline = Pipeline([
        ('preprocess', preprocessor),
        ('logreg', model)
    ])
    return modelwpipeline

#### basic data processing

dataset over or undersampled for speed testing only

In [37]:
df = pd.read_csv(DATASET_PATH, dtype={'PatientId': 'category',
                                      'AppointmentID': 'category',
                                      'Gender': 'category',
                                      'Neighbourhood': 'category',
                                      }, 
                                parse_dates=['ScheduledDay', 
                                             'AppointmentDay'])
df = df.drop(columns=['AppointmentID', 'PatientId'])
n_rows = df.shape[0]
nrepeats,remiainder = NUM_ROWS // n_rows , NUM_ROWS % n_rows
df = pd.concat([df]*nrepeats + [df.sample(remiainder)], ignore_index=True)
display(df.info())
display(df.head())
y = df["Showed_up"]
X = df.drop(columns=["Showed_up"])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 13 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   Gender          1000000 non-null  category      
 1   ScheduledDay    1000000 non-null  datetime64[ns]
 2   AppointmentDay  1000000 non-null  datetime64[ns]
 3   Age             1000000 non-null  int64         
 4   Neighbourhood   1000000 non-null  category      
 5   Scholarship     1000000 non-null  bool          
 6   Hipertension    1000000 non-null  bool          
 7   Diabetes        1000000 non-null  bool          
 8   Alcoholism      1000000 non-null  bool          
 9   Handcap         1000000 non-null  bool          
 10  SMS_received    1000000 non-null  bool          
 11  Showed_up       1000000 non-null  bool          
 12  Date.diff       1000000 non-null  int64         
dtypes: bool(7), category(2), datetime64[ns](2), int64(2)
memory usage: 39.1 M

None

,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,Showed_up,Date.diff
0,F,2016-04-29,2016-04-29,62,JARDIM DA PENHA,False,True,False,False,False,False,True,0
1,M,2016-04-29,2016-04-29,56,JARDIM DA PENHA,False,False,False,False,False,False,True,0
2,F,2016-04-29,2016-04-29,62,MATA DA PRAIA,False,False,False,False,False,False,True,0
3,F,2016-04-29,2016-04-29,8,PONTAL DE CAMBURI,False,False,False,False,False,False,True,0
4,F,2016-04-29,2016-04-29,56,JARDIM DA PENHA,False,True,True,False,False,False,True,0


#### Logistic Regression

In [61]:
# MODEL DEFINITION
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000,solver='lbfgs', class_weight="balanced", n_jobs=N_JOBS_MODEL)
modelp = pipeline(X,model)

# TRAINING
start_time = time.time()
modelp.fit(X, y)
end_time = time.time()
print(f"Training time: {end_time - start_time} seconds")

# SAVING
datetime_now = datetime.datetime.now().strftime("%H%M_%d%m%Y")
model_name = model.__class__.__name__ + f"_{datetime_now}.joblib"
model_save_path = os.path.join(MODELSAVE_PATH, model_name)
joblib.dump(modelp, model_save_path)


Training time: 5.957391023635864 seconds


['c:\\Users\\jjaramil\\OneDrive - InterSystems Corporation\\Documents\\model_parallelization\\models\\LogisticRegression_1150_28112025.joblib']